# Pipeline
- 여러 단계(전처리, 변환, 추정)를 연속적으로 연결하여 실행하는 도구
- 전처리와 모델을 하나의 객체로 결합해서 사용이 가능
- 교차검증, 하이퍼 파라미터 탐색에서 유용 사용
- 데이터의 누수를 방지 : fit_transform()이 학습데이터에만 적용이 되도록 자동 관리

- 매개변수
    - steps
        - 필수 항목 (dict 형태의 데이터 타입)
        - 파이프라인의 단계들을 커스텀한 이름과 객체들을 쌍으로 묶어서 dict 형태로 구성
        - 예. [ ( 'scaler', StandardScaler()), ( 'svc', SVC() ) ]
    - verbose
        - 기본값: False
        - 각 단계가 실행이 될 때 로그를 출력할 것인가?
- 속성
    - named_steps
        - 파이프라인의 각 단계를 딕셔너리형처럼 접근이 가능
        - 예. pipe.named_steps['svc']

- 메서드
    - fit(x, y, fit_params)
        - 순서대로 각 단계의 fit을 진행
        - 마지막 단계는 예측이 가능한 모델이어야한다.
    - fit_transform(x, y, fit_params)
        - 순서대로 각 단계의 fit_transform()을 진행
        - 마지막 단계가 변환을 시켜주는 클래스
    - predict(x)
        - 마지막 단계에서 predict 함수를 호출

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.datasets import load_iris

In [ ]:
# 데이터 로드
iris = pd.read_csv("../data_git/data/iris.csv")

In [ ]:
iris['target'].unique()

In [ ]:
for i, key in enumerate(iris['target'].unique()):
    print(i)
    print(key)
    iris['target'] = iris['target'].replace(key, i)

In [ ]:
iris['target'].value_counts()

In [ ]:
x = iris.drop('target', axis=1)
y = iris['target']

X_train, X_test, Y_train, Y_test = train_test_split(
    x, y, 
    test_size=0.3,
    random_state=42,
    stratify=y
)

In [ ]:
# 파이프라인을 생성 -> StdScaler를 이용, SVC() 모델을 사용
pipe = Pipeline(
    [
        ('Scaler', StandardScaler()),
        ('svc', SVC()),
    ],
    verbose = True
)

In [ ]:
pipe.fit(X_train, Y_train)

In [24]:
pred_pipe = pipe.predict(X_test)

In [26]:
# 분류 보고서 출력
print(classification_report(pred_pipe, Y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       0.93      0.88      0.90        16
           2       0.87      0.93      0.90        14

    accuracy                           0.93        45
   macro avg       0.93      0.93      0.93        45
weighted avg       0.93      0.93      0.93        45



# GridSearchCV
- 하이퍼파라미터(매개변수) 조합을 탐색하여 최적의 조합을 찾는 방법
- 각 파라미터 조합별로 교차검증(CV)을 수행해 평균 성능 비교
- 최적의 모델과 성능을 자동으로 제공

- 매개변수
    - estimator
        - 모델의 선택
        - 예. SVC()
    -param_grid
        - 탐색할 파라미터의 조합
        - 예. {'C' : [0, 1, 10], 'kernel' : ['linear', 'rbf']}
    - cv
        - 교차 검증의 횟수
        - 5 -> 5-폴드 교차 검증
    - scoring
        - 기본값 : None
        - 평가 지표 설정
        - 기존에 제공하는 평가 지표와 커스텀하여 생성한 평가 지표도 사용 가능
    - refit
        - 기본값: True
        - 최적의 파라미터로 전체 데이터를 다시 학습할 것인가
        - True인 경우에는 속성 중 best_estimator_ 사용 가능
    - error_score
        - 기본값: np.nan
        - 모델 학습 시 오류가 발생했을 때 어떤 방식으로 처리할 것인가?
        - "raise" -> 에러 발생/숫자 -> 해당 스코어를 발생하는 숫자로 표시
    - return_train_score
        - 기본값: False
        - 교차검증 시 훈련 성능 점수까지 반영을 할 것인가?
    - verbose
        - 기본값 : 0
        - 출력 로그의 수준 (0: 없음, 1: 간단하게 표시, 2: 상세하게 표시)
    - n_jobs
        - 기본값: None
        - cpu 병렬 처리 개수
        - -1 : 모든 코어를 사용
- 속성
    - cv_results
        - 각 파라미터 조합 별 성능 결과 (훈련/검증 점수, fit 시간)
    - best_estimator_
        - 최적의 파라미터로 다시 학습이 된 모델 객체
    - best_params_
        - 최적의 성능을 낸 파라미터 조합
    - best_score_
        - 최적의 파라미터 조합을 이용한 교차 검증에서의 평균 성능 점수
    - refit_time_
        - 최적의 파라미터로 refit을 하는데 걸린 시간
- 메서드
    - fit(x, y)
        - 모든 파라미터 조합에 대해 학습하고 평가 -> 최적의 모델을 학습
    - predict(x)
        - 최적의 모델을 이용해 예측
    - predict_proba(x)
        - 분류인 경우에 확률 예측
    - score(x, y)
        - 최적의 모델로 점수를 출력

In [34]:
# 독립변수 x, 종속변수 y 그대로 이용
# 파라미터 탐색에서 사용할 모델을 생성
svc = SVC()

# 최적의 파라미터를 찾기 위한 파라미터 조합
param_grid = {
    'C' : [0.1, 1, 10],
    'kernel' : ['linear', 'rbf'],
    'gamma' : ['scale', 'auto']
}

# SVC가 분류모델이니까 -> 평가 지표 -> accuracy
# 교차 검증의 횟수는 5회
grid = GridSearchCV(
    estimator=svc, 
    param_grid = param_grid,
    cv = 5, 
    scoring = 'accuracy',
    verbose = 1,
    refit = True
)
# 학습을 돌려서 최적의 파라미터를 구성한다.
grid.fit(x, y)

# 최적의 파라미터 조합들을 확인
print("최적의 파라미터 조합 : ", grid.best_params_)
print("최고의 score : ", grid.best_score_)
print("최적의 분류 모델 : ", grid.best_estimator_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
최적의 파라미터 조합 :  {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
최고의 score :  0.9800000000000001
최적의 분류 모델 :  SVC(C=1, kernel='linear')
